In [6]:
import os
from pyspark.sql import SparkSession

# Set GCS credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/longnguyen/credentials/key.json'

# Create Spark session with GCS connector
spark = SparkSession.builder \
    .appName("Test GCS with Spark") \
    .master("spark://node-2:7077") \
    .config("spark.jars.packages",
        "io.delta:delta-spark_2.13:4.0.0,"       
        "com.google.cloud.bigdataoss:gcs-connector:hadoop3-2.2.11"
    ) \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/home/longnguyen/credentials/key.json") \
    .getOrCreate()

# Verify connection
print(f"Spark Version: {spark.version}")
print(f"Master: {spark.sparkContext.master}")
print(f"App Name: {spark.sparkContext.appName}")
print("✓ Spark session with GCS connector created successfully!")

:: loading settings :: url = jar:file:/home/longnguyen/miniconda3/envs/spark_env/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/longnguyen/.ivy2.5.2/cache
The jars for the packages stored in: /home/longnguyen/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
com.google.cloud.bigdataoss#gcs-connector added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-963c2ebd-b5a1-414e-85a8-2b16c4e691cc;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
	found com.google.cloud.bigdataoss#gcs-connector;hadoop3-2.2.11 in central
	found com.google.api-client#google-api-client-jackson2;2.0.1 in central
	found com.google.api-client#google-api-client;2.0.1 in central
	found com.google.oauth-client#google-oauth-client;1.34.1 in central
	found com.google.http-c

Spark Version: 4.0.0
Master: spark://node-2:7077
App Name: Test GCS with Spark
✓ Spark session with GCS connector created successfully!


In [7]:
# Write DataFrame to GCS bucket in different formats
# Replace 'your-bucket-name' with your actual GCS bucket name

GCS_BUCKET = "cdc-pipeline-data"  # Change this to your GCS bucket name
GCS_BASE_PATH = f"gs://{GCS_BUCKET}/uat/bronze"

In [9]:
# Verify the files were created by reading them back
print("=== Verifying files in GCS ===\n")


delta_path = f'{GCS_BASE_PATH}/products'
# Read Delta Lake
print("Reading Delta Lake from GCS:")
df_delta = spark.read.format("delta").load(delta_path)
df_delta.show()

print("✓ All files verified successfully!")

=== Verifying files in GCS ===

Reading Delta Lake from GCS:


25/12/22 15:12:41 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+------------+--------+-------------+-------------+-------------+---------+------------+--------------------+------+---------+--------------------+
|product_id|product_name|aisle_id|department_id|cdc_operation|cdc_timestamp|source_db|source_table|     kafka_timestamp|offset|partition| ingestion_timestamp|
+----------+------------+--------+-------------+-------------+-------------+---------+------------+--------------------+------+---------+--------------------+
|      NULL|        NULL|    NULL|         NULL|         NULL|         NULL|     NULL|        NULL|2025-12-21 08:19:...|     0|        2|2025-12-22 14:57:...|
|      NULL|        NULL|    NULL|         NULL|         NULL|         NULL|     NULL|        NULL|2025-12-21 08:19:...|     1|        2|2025-12-22 14:57:...|
|      NULL|        NULL|    NULL|         NULL|         NULL|         NULL|     NULL|        NULL|2025-12-21 08:24:...|     2|        2|2025-12-22 14:57:...|
|      NULL|        NULL|    NULL|         NUL

In [ ]:


# Stop Spark session
spark.stop()
print("✓ Spark session stopped")

✓ Spark session stopped
